In [187]:
from http.client import REQUESTED_RANGE_NOT_SATISFIABLE
import pandas as pd
from sodapy import Socrata
import matplotlib.pyplot as plt
import geo_comm_areas as gca
pd.set_option('display.max_rows', None)


/home/anth0nyhak1m/.local/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
id_dict = {
    "red light violations": "spqx-js37", "abandoned buildings": "kc9i-wq85",
    "socioeconomic indicators": "kn9c-c2s2", "hardship index": "792q-4jtu",
    "crimes": "ijzp-q8t2", "grocery stores": "4u6w-irs9" }



In [3]:
def pull_data(dataset_id, lim):
    '''
    Given a dataset id, pull out data from Chicago portal API

    Inputs:
      dataset_id: (str) a unique id for one Chicago portal dataset
      limit: (int) number of rows to pull each time, default to None

    Return:
      (Pandas DataFrame) a dataframe pulled from Chicago Portal
    '''
    client = Socrata("data.cityofchicago.org", "goD601SLndI51xcMq1KsnG6np")

    results = client.get(dataset_id, limit = lim)
    return pd.DataFrame.from_records(results)

In [4]:
def gen_tables(id_dictionary, lim):
    dataset_dct = {}
    for dataset_name, url in id_dictionary.items():
        dataset_dct[dataset_name] = pull_data(url, lim)

    return dataset_dct

In [ ]:
dct = gen_tables(id_dict, 1000)



In [ ]:
for key in dct.keys():
    print(key)

In [117]:


client = Socrata("data.cityofchicago.org", "goD601SLndI51xcMq1KsnG6np")
results = client.get("ijzp-q8t2", where = f"(primary_type) = 'HOMICIDE'",
                    limit=1000000)

crime_df = pd.DataFrame.from_records(results)


In [ ]:

crime_df.columns.unique()

In [138]:
crime_year_count = crime_df.groupby(by=['community_area', "year"]).size().unstack()




In [ ]:
crime_year_count = crime_year_count.fillna(0)

crime_year_count['Average Homicide'] = crime_year_count.mean(axis=1)

crime_year_count['community area'] = crime_year_count.index

x = crime_year_count.melt('community area')

x


In [131]:
type(x)

pandas.core.frame.DataFrame

In [188]:
gca.get_ca_census()

,area_num,comm_name,tot_pop,pop_hh,pop_gq,hisp,white,black,asian,other,total_homes,tot_hh,vac_homes,hh_size,vac_rate,hisp_per,white_per,black_per,asian_per
0,1,Rogers Park,55628,52242,3386,12587,22292,13819,3820,3110,28531,26402,2129,1.978714,7.462059,22.627094,40.073344,24.841806,6.867045
1,2,West Ridge,77122,75531,1591,15307,28212,10484,19815,3304,28249,26493,1756,2.850980,6.216149,19.847774,36.581002,13.594046,25.693058
2,3,Uptown,57182,55082,2100,7587,29871,10800,6182,2742,35019,32215,2804,1.709825,8.007082,13.268161,52.238467,18.887062,10.811094
3,4,Lincoln Square,40494,39969,525,7069,25342,1601,4516,1966,20431,19143,1288,2.087917,6.304146,17.456907,62.582111,3.953672,11.152269
4,5,North Center,35114,35058,56,4503,25799,780,2162,1870,15936,14931,1005,2.348001,6.306476,12.823945,73.472119,2.221336,6.157088
5,6,Lake View,103050,102745,305,9340,77133,3917,7717,4943,61920,57721,4199,1.780028,6.781331,9.063561,74.850073,3.801067,7.488598
6,7,Lincoln Park,70492,67694,2798,4928,54378,2895,5169,3122,38649,35570,3079,1.903121,7.966571,6.990864,77.140668,4.106849,7.332747
7,8,Near North Side,105481,103776,1705,6587,73970,8576,12313,4035,77429,66685,10744,1.556212,13.875938,6.244727,70.126373,8.130374,11.673192
8,9,Edison Park,11525,11524,1,1239,9538,62,308,378,4960,4733,227,2.434819,4.576613,10.750542,82.759219,0.537961,2.672451
9,10,Norwood Park,38303,37893,410,6247,28208,363,2222,1263,16035,15323,712,2.472949,4.440287,16.309427,73.644362,0.947706,5.801112


In [196]:
grocery_stores = dct['grocery stores']

store_name = dct['grocery stores']['store_name']

#dct['grocery stores'].columns.unique()

grocery_stores['store_name'].str.contains('LIQUOR').value_counts()[True]

groc_count = dct['grocery stores'].groupby(by=['community_area']).size().to_frame(name='grocery stores count')

liquor_stores = grocery_stores[grocery_stores['store_name'].str.contains('LIQUOR')]

liquor_count = liquor_stores.groupby(by=['community_area']).size().to_frame(name='liquor stores count')

groceries_df = groc_count.merge(liquor_count, left_on = "community_area", right_on = "community_area")

groceries_df['Drunk Ratio'] = groceries_df['liquor stores count']/groceries_df['grocery stores count'] * 100

groceries_df['year'] = '2011'

#groceries_df.melt('community area')

KeyError: 'community area'

In [ ]:
dct['hardship index'].columns.unique()
dct['socioeconomic indicators']


Grocery stores: 
1. Goals
    a. columns: comm area, if possible comm area name, total groceries stores, num of liquor store, liquor/total gorcery stores ratio, year (??) - Sasha

Crime: Status Done
1. Goals
    a. columns: comm area, if possible comm area name, homicide count, avg homicide rate (total homicide/no. of years) - Anthony

socioeconomic indicators
1. Goals
    a. columns: comm_area (make it a number), comma_area name, hardship index, year - Anthony